In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import sklearn.metrics as sk
import glob

pd.options.mode.chained_assignment = None 

In [ ]:
path = "C:/Users/Isabel Cristina/Desktop/Set_proyection/Backtest_launcher/BT/*.htm" 
reports = glob.glob(path)


magic_number = reports[3].split()[2]
print('Reading BT: ', magic_number)
file = pd.read_html(reports[3])
backtest_df = file[1]
backtest_df.columns = backtest_df.iloc[0]
backtest_df = backtest_df.iloc[1:]
backtest_df = backtest_df[backtest_df['Balance'].notna()].reset_index(drop=True)
backtest_df['Magic_Number'] = magic_number
bt_df = backtest_df
bt_df['Tiempo'] = pd.to_datetime(bt_df['Tiempo'])
bt_df['Beneficios'] = pd.to_numeric(bt_df.Beneficios)
bt_df['Balance'] = pd.to_numeric(bt_df.Balance)

In [ ]:
bt_df

In [ ]:
#Beneficio bruto
ben_bruto = bt_df[bt_df.Beneficios > 0]['Beneficios'].sum()

#Perdida bruta
per_bruta = bt_df[bt_df.Beneficios < 0]['Beneficios'].sum()

#Profit factor 
pf = ben_bruto/abs(per_bruta)

#Beneficio esperado
ben_esp = (ben_bruto+per_bruta)/len(bt_df)

#Maximo DD
dd_df = bt_df.copy()
dd_df = dd_df[['Tiempo','Balance']]
dd_df = pd.DataFrame({'Tiempo': [dd_df.Tiempo[0]], 'Balance': [10000]}).append(dd_df).reset_index()
dd_df['Act_max'] = dd_df.Balance.cummax()
dd_df['DD'] = dd_df.Act_max - dd_df.Balance
max_dd = dd_df['DD'].max()

#Maximo beneficio
max_ben = bt_df.Beneficios.sum()

#Net recovey
net_factor = max_ben/max_dd

#CA 
month_df = bt_df.copy()
month_df = month_df[['Tiempo']]
month_df['month_year'] = month_df['Tiempo'].dt.to_period('M')
month_group = month_df.groupby(['month_year']).count()
total_months = len(month_group)
ca_kpi = max_ben/ (max_dd * total_months)

#Divergencia de ganadoras
mean_sl = abs(bt_df[bt_df.Beneficios < 0]['Beneficios'].mean())
mean_tp = bt_df[bt_df.Beneficios > 0]['Beneficios'].mean()
min_div = mean_sl/(mean_tp+mean_sl)
neg_trades = len(bt_df[bt_df.Beneficios < 0])
pos_trades = len(bt_df[bt_df.Beneficios > 0])
real_div = pos_trades/(pos_trades+neg_trades)

#R-squared
r2_df = bt_df[['Tiempo', 'Balance']].set_index('Tiempo')
r2_df['Time_num'] = mdates.date2num(r2_df.index)
reg = np.polyfit(np.array(r2_df.Time_num),np.array(r2_df.Balance),1)
r2_df['Regre'] = reg[0]*r2_df.Time_num + reg[1]
r2_kpi = sk.r2_score(r2_df.Balance, r2_df.Regre)
r2_kpi

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=r2_df.index,
                y=r2_df.Balance,
                line_color = 'yellow'))
fig.add_trace(go.Scatter(
                x=r2_df.index,
                y=r2_df.Regre,
                line_color = 'black'))

In [ ]:
dd_df2 = dd_df.copy()
bt_df2 = bt_df.copy()

In [ ]:
dd_tot = bt_df.append(bt_df2).reset_index()
dd_tot = dd_tot[['Tiempo','Beneficios']].sort_values('Tiempo')
dd_tot['Balance'] = dd_tot.Beneficios.cumsum()
dd_tot = pd.DataFrame({'Tiempo': [dd_tot.Tiempo[0]], 'Balance': [0]}).append(dd_tot).reset_index()
dd_tot['Act_max'] = dd_tot.Balance.cummax()
dd_tot['DD'] = dd_tot.Act_max - dd_tot.Balance
max_dd_tot = dd_tot['DD'].max()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                x=dd_df.Tiempo,
                y=dd_df.DD,
                opacity=0.5,
                fill='tozeroy'))
fig.add_trace(go.Scatter(
                x=dd_df2.Tiempo,
                y=dd_df2.DD,
                opacity=0.5,
                fill='tozeroy'))
fig.add_trace(go.Scatter(
                x=dd_tot.Tiempo,
                y=dd_tot.DD,
                line_color = 'black'))

***Monte Carlo con Condiciones***

In [ ]:
data_mt = data_df[data_df.Magic_Number == '19095']
total_trades = len(data_mt)
df_real = data_mt.loc[:, 'Beneficios'].to_frame()

In [ ]:
fig = px.box(df_real, y = 'Beneficios')
fig.show()

In [ ]:
#random.seed(30)
#flags and configs
delete_percentile = True
select_perncentile = 95 # value between 0 and 100
decrease_beneficio = False
decrease_value = 0.90 #value betwen 0 and 1

it_num = 100
#calculation
df_percent = df_real

if (delete_percentile):
    percent_value = np.percentile(df_real,select_perncentile)
    df_percent = df_real.loc[df_real.Beneficios <= percent_value,].reset_index(drop=True)
    print('Percentile value : {:0.2f}'.format(percent_value))

if (decrease_beneficio) :
    decre_list = pd.Series(np.random.randint(2, size=len(df_percent)), name = 'dummy_decre')
    df_percent = pd.concat([df_percent, decre_list], axis = 1)
    df_percent['Beneficios'] = df_percent.Beneficios-(df_percent.dummy_decre*(df_percent.Beneficios*decrease_value))

# iteration calculations
it_dic = {it: df_percent.loc[random.choices(list(df_percent.index), k = total_trades),'Beneficios'].reset_index()
       for it in range(1,it_num)}

df_it  = pd.concat(it_dic.values(), axis = 1)
df_it = df_it.drop('index',1)
df = pd.concat([df_real, df_it], axis=1)
df = df.cumsum()

print('Ganancia del set: {:0.2f}'.format(df.iloc[-1,0]))

#plot
palette = plt.get_cmap('Set1')
num=0
plt.figure(figsize=(12,8))
for column in df.columns:
    num+=1
    plt.plot(df.index, df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

***Prueba Rolling Profic Factor***

In [ ]:
# data_df = data_df[data_df.Magic_Number =='22058']
cesta = magic_list
data_df = data_df[data_df.Magic_Number.isin(cesta)]

data_porfit_factor = pd.DataFrame()

for magic in cesta:

    data_aux = data_df[data_df.Magic_Number ==magic]
    data_prueba = data_aux.sort_values(by=['Tiempo'])
    data_prueba['Beneficios_pos'] = data_prueba[data_prueba.Beneficios > 0]['Beneficios']
    data_prueba['Beneficios_pos'] = data_prueba['Beneficios_pos'].fillna(0)
    data_prueba['Beneficios_neg'] = data_prueba[data_prueba.Beneficios < 0]['Beneficios']
    data_prueba['Beneficios_neg'] = data_prueba['Beneficios_neg'].fillna(0)
    data_prueba['Profit_cum_pos'] = 0
    data_prueba['Profit_cum_neg'] = 0
    data_prueba['Window'] = 0

#     for i in range(5,31,5):

#         data_prueba['Profit_cum_pos'] = data_prueba['Beneficios_pos'].rolling(i).sum()
#         data_prueba['Profit_cum_neg'] = data_prueba['Beneficios_neg'].rolling(i).sum()
#         data_prueba['Window'] = i
#         data_porfit_factor = data_porfit_factor.append(data_prueba)

    roll_window = round(cons_trades_neg[magic]*1.3+0.5)
    data_prueba['Profit_cum_pos'] = data_prueba['Beneficios_pos'].rolling(roll_window).sum()
    data_prueba['Profit_cum_neg'] = data_prueba['Beneficios_neg'].rolling(roll_window).sum()
    data_prueba['Window'] = roll_window
    data_porfit_factor = data_porfit_factor.append(data_prueba)
    

data_porfit_factor = data_porfit_factor.replace([np.inf, -np.inf], 0)
data_porfit_factor = data_porfit_factor[data_porfit_factor['Profit_cum_pos'].notna()].reset_index(drop=True)
data_porfit_factor['Profit_Factor_roll'] = (data_porfit_factor['Profit_cum_pos'] / data_porfit_factor['Profit_cum_neg']).abs()

data_porfit_factor = data_porfit_factor.drop(['Beneficios_pos', 'Beneficios_neg'],axis=1)

In [ ]:
df_backtest[df_backtest.Magic_Number.isin(selected_port)]

In [ ]:
windows_list = [*range(5,31,5)]

color = []
for i in range(1,(len(magic_in_real)+1)):
    color.append('#%06X' % randint(0, 0xFFFFFF))
# for i in windows_list:
#     color.append('#%06X' % randint(0, 0xFFFFFF))
    

***Profit Factor por Ventana***

In [ ]:
dd_prueba = data_porfit_factor[ data_porfit_factor['Profit_Factor_roll'] <= 6]

fig = px.box(dd_prueba, x="Window", y="Profit_Factor_roll", color="Magic_Number")
fig.show()

In [ ]:
from plotly.offline import plot
plot(fig, 
     filename='test.html')

In [ ]:
dd_prueba.describe()

In [ ]:
fig = go.Figure()


for i in range(0,len(magic_in_real)):
    
    df_filter = data_porfit_factor[data_porfit_factor.Magic_Number == magic_in_real[i]]
    df_filter = df_filter[(df_filter.Profit_Factor_roll <= 10)]
    
    fig.add_trace(go.Scatter(
                    x=df_filter.Tiempo,
                    y=df_filter.Profit_Factor_roll,
                    name=str(magic_in_real[i]),
                    line_color=color[i],
                    opacity=0.8))

fig.show()